In [1]:
# conda env: clamp_env (Python 3.8.0)
import os
import sys
from datacat4ml.const import DATA_DIR, FIG_DIR, FETCH_DATA_DIR , FETCH_FIG_DIR

import torch
import clamp

import pandas as pd

In [2]:
# load the dataset
or_chembl_id = ['CHEMBL233', 'CHEMBL237', 'CHEMBL236', 'CHEMBL2014']
gpcr_ki = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ki_maxcur_8_data.csv'))
# extract the rows where the 'target_chembl_id' is one of the elements in the list OR_chembl_id
or_ki = gpcr_ki[gpcr_ki['target_chembl_id'].isin(or_chembl_id)]

/tmp/ipykernel_3630258/3583070045.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  gpcr_ki = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ki_maxcur_8_data.csv'))


In [31]:
or_ki.pchembl_value

0         6.96
5         5.97
6         4.97
7         5.39
8         4.41
          ... 
139389    6.13
139402    5.93
139403    5.93
139412    6.70
139413    6.70
Name: pchembl_value, Length: 13533, dtype: float64

In [32]:
or_ki_mols = or_ki.canonical_smiles.to_list()
or_ki_mols

['CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](NC(=O)CNC(=O)CCC(=O)NCC(=O)N[C@H]2CC[C@@]4(O)[C@H]6Cc7ccc(O)c8c7[C@@]4(CCN6C)[C@H]2O8)CC[C@@]3(O)[C@H]1C5',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](NC(=O)CNC(=O)CNC(=O)CNC(=O)CCC(=O)NCC(=O)NCC(=O)NCC(=O)N[C@H]2CC[C@@]4(O)[C@H]6Cc7ccc(O)c8c7[C@@]4(CCN6C)[C@H]2O8)CC[C@@]3(O)[C@H]1C5',
 'O=C(CCC(=O)NCC(=O)N[C@@H]1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3CC2CC2)[C@H]1O5)NCC(=O)N[C@@H]1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3CC2CC2)[C@H]1O5',
 'Cc1ccc(C(c2ccc(C)cc2)N2CC[C@H]2[C@H](N)c2cccc(Cl)c2)cc1',
 'Cc1ccc(C(c2ccc(C)cc2)N2CC[C@H]2[C@H](N)c2cccc(Cl)c2)c

In [33]:
or_ki_assay_descs = or_ki.assay_desc.to_list()
or_ki_assay_descs

['Displacement of [3H]EK from Opioid receptor delta 1 in guinea pig brain membrane',
 'Binding affinity at human ORL1 receptor was determined by using [125I]nociceptin radioligand in Chinese hamster ovary (CHO) cell membranes at a concentration of 23 pM',
 'Binding affinity at human opioid receptor mu 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.12 nM',
 'Binding affinity at human opioid receptor kappa 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.31 nM',
 'Binding affinity at human opioid receptor delta 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.8 nM',
 'Binding affinity at human ORL1 receptor was determined by using [125I]nociceptin radioligand in Chinese hamster ovary (CHO) cell membranes at a concentration of 23 pM',
 'Binding affinity at human opioid receptor mu 1 was determined by using [3H]diprenorphine radiol

In [34]:
model = clamp.CLAMP(device='cpu')
model.eval()

2024-10-22 12:59:39.074 | INFO     | clamp.models.pretrained:__init__:33 - Loaded pretrained model from data/models/clamp_clip/checkpoint.pt


PretrainedCLAMP(
  (compound_encoder): NetworkLayerNorm(
    (linear_input): Linear(in_features=8192, out_features=4096, bias=True)
    (linear_hidden_l): ModuleList(
      (0): Linear(in_features=4096, out_features=2048, bias=True)
    )
    (linear_output): Linear(in_features=2048, out_features=768, bias=True)
    (normalization_input): LayerNorm((4096,), eps=1e-05, elementwise_affine=False)
    (normalization_hidden_l): ModuleList(
      (0): LayerNorm((2048,), eps=1e-05, elementwise_affine=False)
    )
    (nonlinearity): ReLU()
    (dropout_input): Dropout(p=0.1, inplace=False)
    (dropout_hidden): Dropout(p=0.2, inplace=False)
  )
  (assay_encoder): NetworkLayerNorm(
    (linear_input): Linear(in_features=512, out_features=4096, bias=True)
    (linear_hidden_l): ModuleList(
      (0): Linear(in_features=4096, out_features=2048, bias=True)
    )
    (linear_output): Linear(in_features=2048, out_features=768, bias=True)
    (normalization_input): LayerNorm((4096,), eps=1e-05, elem

In [39]:
molecules = or_ki_mols[:5]
assay_descriptions = [or_ki_assay_descs[0]]

In [40]:
molecules

['CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](NC(=O)CNC(=O)CCC(=O)NCC(=O)N[C@H]2CC[C@@]4(O)[C@H]6Cc7ccc(O)c8c7[C@@]4(CCN6C)[C@H]2O8)CC[C@@]3(O)[C@H]1C5',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1']

In [41]:
with torch.no_grad():
    logits = model.forward_dense(molecules, assay_descriptions)
    probs = logits.softmax(dim=0).cpu().numpy() # probs for molecules

print("Mol probs for assay:", probs[:,0])

Mol probs for assay: [0.19077389 0.20230654 0.20230654 0.20230654 0.20230651]


In [42]:
probs

array([[0.19077389],
       [0.20230654],
       [0.20230654],
       [0.20230654],
       [0.20230651]], dtype=float32)